In [1]:
import streamlit as st
from langchain.llms import OpenAI
import tiktoken
import os
import numpy as np
import json




st.title('UAL Flight Review Sentiment Analysis')
 
openai_api_key = st.sidebar.text_input('OpenAI API Key')

from langchain.llms import OpenAI
global llm
llm = None
global revdata
revdata = None

# Open the JSON file
with open("All_Reviews_Processed.json", "r") as f:
    revdata = json.load(f)  

    
    
# Prompt Start Lines, Intermediate Lines, End Lines and Questions
sl1 = "Following is a review of United Airlines flight customer:"
q1 = "Is the above text written by a flight customer a review or a feedback. Does it have any negative or positive words? Does it have any concerns explained? Answer me in 'YES' or 'NO' if it is some review"
q2 = "Answer me on a scale of -5 to 5 how good or bad this review is. -5 means very negative. 5 means very positive. Answer only in single number."
q3 = "Can you parse the above review into multiple sentences such that each sentence is complete in itself, talks about one issue and no pronouns are used? Dont use new adjectives other than mentioned in the original review. Dont exaggerate. Provide a numbered list of sentences. Do not create new sentences on your own. Use the parts from the original review."
q4 = "Can you split the above review into multiple sentences without using any pronouns and generate complete sentences?"
q5 = "Is this review talking about ASPECT? Answer me in YES or NO."
q6 = "Can you correct the spelling mistakes in the above review? If there are no spelling mistakes, just print the same exact review" 
q7 = "Can you rewrite the same sentence with coreferences resolved?"

# FEW SHOT PROMPTING

# SAMPLE EXAMPLE
examplelist = ["The food was bad and dirty Airplane was awful, looked awful",
               "Food was bad, flight delayed, poor check in"]
outputlist = [
'''
1. The food was bad and dirty
2. Airplane was awful, looked awful
''',
'''
1. Food was bad
2. Flight delayed
3. Poor check in
'''
]
sl = "Following are the " + str(len(examplelist)) + " examples:"
examplesuppline = "It should be splitted into following two sentences:"
ml = "Given the following review:"
finalq = "Can you parse the above review into multiple parts such that each sentence is complete in itself. Provide a numbered list of sentences."
placeholderkey = "#REVIEW#"


def create_few_shot_prompt(sl, examplelist, outputlist, ml, placeholderkey, finalq):
    """
    Creates a prompt for few-shot learning with placeholders for examples and output.

    Args:
    - sl (str): Starting line for the prompt.
    - examplelist (list of str): List of example texts.
    - outputlist (list of str): List of corresponding outputs for examples.
    - ml (str): Middle line for the prompt.
    - placeholderkey (str): Placeholder key to be replaced with text.
    - finalq (str): Final question for the prompt.

    Returns:
    - str: The constructed prompt.
    """
    prompt = ""
    prompt += sl + "\n"
    for i, each in enumerate(examplelist):
        prompt = prompt + str(i+1) + ". " + "'" + str(examplelist[i]) + "'" + '\n'
        prompt = prompt + examplesuppline + "\n"
        prompt = prompt + str(outputlist[i]) + "\n\n"

    prompt= prompt + ml + "\n" + placeholderkey + "\n\n"
    prompt = prompt + finalq
    return prompt

def create_final_prompt(placeholderprompt, text):
    """
    Creates the final prompt by replacing a placeholder with provided text.

    Args:
    - placeholderprompt (str): Prompt containing a placeholder.
    - text (str): Text to replace the placeholder.

    Returns:
    - str: The final prompt with the placeholder replaced by text.
    """
    prompt = placeholderprompt.replace(placeholderkey, text)
    return prompt


2024-05-08 18:54:03.550 
  command:

    streamlit run C:\Users\saswat.swain\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

def get_response(prompt, technology="gpt4_v1"):
    """
    Generates a response to a given prompt using the specified language model technology.

    Args:
    - prompt (str): The prompt to generate a response for.
    - technology (str): The technology or model to use for generating the response (default is "gpt4_v1").

    Returns:
    - str: The generated response.
    """
    if technology == "gpt4_v1":
        global llm
        op = llm.invoke(
            [
                HumanMessage(
                    content=prompt
                )
            ]
        )  
        return op.content

def sentence_splitting(review):
    """
    Splits a given review into multiple sentences using few-shot learning.

    Args:
    - review (str): The review text to be split into sentences.

    Returns:
    - str: The response containing the split sentences or the original text if splitting fails.
    """
    examplelist = ["The food was bad and dirty Airplane was awful, looked awful",
               "Food was bad, flight delayed, poor check in"]
    outputlist = [
    '''
    1. The food was bad and dirty
    2. Airplane was awful, looked awful
    ''',
    '''
    1. Food was bad
    2. Flight delayed
    3. Poor check in
    '''
    ]
    sl = "Following are the " + str(len(examplelist)) + " examples:"
    examplesuppline = "It should be splitted into following two sentences:"
    ml = "Given the following review:"
    finalq = "Can you parse the above review into multiple parts such that each sentence is complete in itself. Provide a numbered list of sentences. If you cannot parse, just print the original text."
    placeholderkey = "#REVIEW#"
    phprompt = create_few_shot_prompt(sl, examplelist, outputlist, ml, placeholderkey, finalq)
    prompt = create_final_prompt(phprompt, review)
    resp = get_response(prompt)
    return resp


In [3]:
def is_it_a_review(review):
    """
    Determines whether the given text is a review or not using few-shot learning.

    Args:
    - review (str): The text to be evaluated.

    Returns:
    - str: "YES" if the text is considered a review, otherwise "NO".
    """
    prompt = "'" + review + "'" + "\n" + q1
    resp = get_response(prompt)
    if "yes" in resp.lower():
        return "YES"
    return "NO"

def remove_typos(review):
    """
    Attempts to remove typos from the given review using few-shot learning.

    Args:
    - review (str): The review text to be corrected.

    Returns:
    - str: The corrected review text.
    """
    prompt = "'" + review + "'" + "\n" + q6
    resp = get_response(prompt)
    return resp


In [4]:
def resolve_coreference_resolution(review):
    """
    Resolves coreference resolution in the given review text using few-shot learning.

    Args:
    - review (str): The review text with potential coreferences to be resolved.

    Returns:
    - str: The review text with resolved coreferences.
    """
    prompt = "'" + review + "'" + q7
    resp = get_response(prompt)
    return resp

def clean_review(singlelinereview):
    """
    Cleans up a review text by removing newline characters and quotes.

    Args:
    - singlelinereview (str): The review text to be cleaned.

    Returns:
    - str: The cleaned review text.
    """
    review = singlelinereview.replace("\n","")
    review = review.replace('"',"")
    return review


In [5]:
def contains_questions(review):
    """
    Determines whether the given text contains questions and extracts them if it does.

    Args:
    - review (str): The text to be evaluated.

    Returns:
    - str: "NO" if the text does not contain questions, otherwise a numbered list of questions.
    """
    que = "Does the above text contain any questions? Answer in YES or NO. If YES, print the numbered list of questions as well."
    prompt = "'" + review + "'" + "\n" + que
    resp = get_response(prompt)
    if " no " in resp.lower():
        return "NO"
    return resp

def contains_comparisons(review):
    """
    Determines whether the flight customer in the given text is making comparisons with other airlines and extracts them if so.

    Args:
    - review (str): The text to be evaluated.

    Returns:
    - str: "NO" if the flight customer is not making comparisons, otherwise a numbered list of comparisons.
    """
    que = "Does the flight customer in above text doing any comparisons of United Airlines with other airlines? Answer in YES or NO. If YES, print the comparisons in a numbered list."
    prompt = "'" + review + "'" + "\n" + que
    resp = get_response(prompt)
    if " no " in resp.lower():
        return "NO"
    return resp


In [6]:
def assign_sentiment_score(sentexamplelist, sentoutputlist, review):
    """
    Assigns a sentiment score to the given review based on examples and output scores.

    Args:
    - sentexamplelist (list of str): List of example sentences.
    - sentoutputlist (list of int): List of corresponding sentiment scores for examples.
    - review (str): The review text to assign a sentiment score to.

    Returns:
    - int: The assigned sentiment score for the review.
    """
    sl = "Following are the " + str(len(sentexamplelist)) + " examples:"
    examplesuppline = "It should be assigned the following sentiment score:"
    ml = "Given the following review:"
    finalq = q2
    placeholderkey = "#REVIEW#"
    phprompt = create_few_shot_prompt(sl, sentexamplelist, sentoutputlist, ml, placeholderkey, finalq)
    prompt = create_final_prompt(phprompt, review)

    # Send the prompt to the language model and extract the sentiment score from the response
    prompt = "'" + review + "'" + "\n" + q2
    resp = get_response(prompt)
    resp = "".join(c for c in resp if not c.isalpha() and c not in [',','.',' '])
    sentscore = int(resp)
    return sentscore


In [7]:
def rev_splits(revsplits):
    """
    Splits a review into subreviews and checks if each subreview is itself a review.

    Args:
    - revsplits (str): The review text containing subreviews separated by newline characters.

    Returns:
    - dict: A dictionary where keys are subreviews and values are 100 if the subreview is a review, otherwise 0.
    """
    # OBTAINED THE LIST OF SUBREVIEWS FROM A REVIEW
    revsplitlist = revsplits.split("\n")
    new1revsplitlist = []
    for each in revsplitlist:
        if len(each) != 0:
            subrev = each.split(". ")[1]
            subrev = clean_review(subrev)
            new1revsplitlist.append(subrev)
    new2revsplitlist = {}
    for each in new1revsplitlist:
        # CHECKING IF THE SUBREVIEW IS A REVIEW 
        prompt = "'" + each + "'" + "\n" + q1
        resp = get_response(prompt)
        if "yes" in resp.lower():
            new2revsplitlist[each] = 100
        else:
            new2revsplitlist[each] = 0
    return new2revsplitlist


In [9]:
def revanalysis(revsplits, revanalysis):
     """
    Analyzes subreviews extracted from a review and assigns aspects and sentiment scores to each subreview.

    Args:
    - revsplits (str): The review text containing subreviews separated by newline characters.
    - revanalysis (dict): The dictionary containing analysis results to be updated.

    Returns:
    - dict: The updated dictionary containing analysis results.
    """
    revanalysis['Sub_Reviews'] = {}
    aspectlist = ["FOOD", "FLIGHT TIMING", "FLIGHT BOARDING", "FLIGHT BAGGAGE", "Flight CHECK IN", "AIRPORT_EXPERIENCE", 
                  "AIRPLANE EXPERIENCE", "FLIGHT SERVICE", "LOYALTY PROGRAM"]
    #Touchpoints in place of Aspects
    #Dont call out anything which look like maintainable
    # Business OP vs Technical OP
    new2revsplitlist = rev_splits(revsplits)
    
    for each1 in new2revsplitlist.keys():
        revanalysis['Sub_Reviews'][each1] = {}
        revanalysis['Sub_Reviews'][each1]['Aspects'] = []
        revanalysis['Sub_Reviews'][each1]['Sentiment Score'] = 0
        if new2revsplitlist[each1] == 0:
            revanalysis['Sub_Reviews'][each1]['Aspects'].append("Others")
            revanalysis['Sub_Reviews'][each1]['Sentiment Score'] = 0
            continue
        aspecttagged = 0
        for each2 in aspectlist:
            prompt = "'" + each1 + "'" + "\n" + q5
            prompt = prompt.replace("ASPECT", each2)
            resp = get_response(prompt)
            #print(prompt)
            #print(resp)
            #print("\n")
            if "yes" in resp.lower():
                # IDENTIFYING SENTIMENT SCORE
                sentscore = assign_sentiment_score(sentexamplelist, sentoutputlist, each1)
                revanalysis['Sub_Reviews'][each1]['Aspects'].append(each2)
                revanalysis['Sub_Reviews'][each1]['Sentiment Score'] = sentscore
                aspecttagged = 1
        if aspecttagged == 0:
            # IDENTIFYING SENTIMENT SCORE
            sentscore = assign_sentiment_score(sentexamplelist, sentoutputlist, each1)
            revanalysis['Sub_Reviews'][each1]['Aspects'].append("No Category")
            revanalysis['Sub_Reviews'][each1]['Sentiment Score'] = sentscore
    return revanalysis

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [10]:
def process_review(review):
    """
    Processes a review by analyzing its aspects, sentiment score, and resolving coreferences.

    Args:
    - review (str): The review text to be processed.

    Returns:
    - dict: A dictionary containing the processed review information including aspects, sentiment score, and resolved coreferences.
    """
    op = {}
    op['review'] = review
    if len(review) < 20:
        flag = is_it_a_review(review)
        op['is_review'] = flag
        if flag == "NO":
            op['sentiment_score'] = 0
            return op
    op['is_review'] = "Yes"
    op['sentiment_score'] = assign_sentiment_score(sentexamplelist, sentoutputlist, review)
    review = remove_typos(review)
    review = resolve_coreference_resolution(review)
    revssplits = sentence_splitting(review)
    finalop = revanalysis(revssplits, op)
    return finalop


In [11]:
def generate_response(input_text):
    """
    Generates a response based on the input text, which can be either a review or a non-review text.

    Args:
    - input_text (str): The input text to generate a response for.

    Returns:
    - str: The generated response.
    """
    try:
        global llm
        os.environ['OPENAI_API_KEY'] = openai_api_key
        #llm = OpenAI(model_name='gpt-4', temperature=0.9, openai_api_key=openai_api_key)
        llm = ChatOpenAI(model="gpt-4", temperature=0.2, logprobs=True)
        if input_text in revdata.keys():
            response = revdata[input_text]['op1']
        else:
            response = process_review(input_text)
        if response['is_review'] == "NO":
            st.write("The above text is NOT A REVIEW")
            st.write("\n\n\nSentiment Score: 0")
        else:
            st.write("The above text is A REVIEW\n\n")
            st.write("Sentiment Score: ", str(response['sentiment_score']))
            st.write("\n\n\n\nSub Sentences:\n\n")
            for i,each in enumerate(response['Sub_Reviews'].keys()):
                st.write(str(i+1) + ". " + each)
                st.write("Touchpoints Tagged: ", response['Sub_Reviews'][each]['Aspects'])
                st.write("Sentiment Score: ", int(response['Sub_Reviews'][each]['Sentiment Score']))
                st.write("\n\n")
    except Exception as e:
        st.error(f"An error occurred: {str(e)}")


In [12]:
def questions_and_comparisons(input_text):
    """
    Analyzes input text for questions and comparisons.

    Args:
    - input_text (str): The input text to analyze.

    Returns:
    - str: The analysis results including questions and comparisons found in the input text.
    """
    try:
        global llm
        os.environ['OPENAI_API_KEY'] = openai_api_key
        #llm = OpenAI(model_name='gpt-4', temperature=0.9, openai_api_key=openai_api_key)
        llm = ChatOpenAI(model="gpt-4", temperature=0.2, logprobs=True)
        if input_text in revdata.keys():
            response1 = revdata[input_text]['op2']['questions']
            response2 = revdata[input_text]['op2']['comparisons']
            st.write(response1)
            st.write("\n\n")
            st.write(response2)
        else:
            response1 = contains_questions(input_text)
            response2 = contains_comparisons(input_text)
            if response1 == "NO":
                response1 = "No Questions Found !!"
            if response2 == "NO":
                response2 = "No Comparisons Found !!"
            response1 = response1.replace("YES", "Questions: ")
            response2 = response2.replace("YES", "Comparisons: ")
            st.write(response1)
            st.write("\n\n")
            st.write(response2)
    except Exception as e:
        st.error(f"An error occurred: {str(e)}")


In [13]:
# Starting a form using Streamlit's form functionality
with st.form('my_form'):
    # Adding a text area input field for entering flight reviews
    text = st.text_area('Enter your flight review:', '')
    # Adding a submit button for generating sentences and sentiment scores
    submitted1 = st.form_submit_button('Sentences & Sentiment Scores')
    # Adding a submit button for analyzing questions and comparisons
    submitted2 = st.form_submit_button('Questions & Comparisons')
    
    # Displaying a warning if OpenAI API key is not provided
    if not openai_api_key.startswith('sk-'):
        st.warning('Please enter your OpenAI API key!', icon='⚠')
    
    # Checking if the Sentences & Sentiment Scores button is submitted and OpenAI API key is provided
    if submitted1 and openai_api_key.startswith('sk-'):
        # Calling the function to generate response for the entered review
        generate_response(text)
    
    # Checking if the Questions & Comparisons button is submitted and OpenAI API key is provided
    if submitted2 and openai_api_key.startswith('sk-'):
        # Calling the function to analyze questions and comparisons in the entered text
        questions_and_comparisons(text)


In [ ]:
allrevlist = [
    "Flight was on time.",
    "It was crowded and hot and humidity was awful. The gate area didn't have enough seats and the flight was delayed and your customer were very uncomfortable",
    "The food was awful it looked awful and tasted awful I will be sending a letter and will attach pictures", 
    "Crapes and pasta",
    "I live united but this trip I was disappointed", 
    "On time departure and arrival, effective boarding.\n\nNegative one restroom was out of order.\n", 
    "Plane was delayed, had smell, very congested",
    "Service good, loyalty program has declined.  I had 27 segments last year and barely made Silver Premier status…PQM note representative of miles traveled.",
    "Should have better options and greater generosity in sharing packages ( say, how about giving 2?)",
    "Seat comfort, efficiency of boarding, on time departure and early arrival, great WiFi,  rapid delivery of checked bag",
    "No. Staff was very helpful kind courteous, and the plane was on time only issues. The seats are very uncomfortable and I’m not a large person.",
    "The smoothest air travel I have experienced in quite some time.  All United personnel were polite and helpful while my flights were on time both for departure and arrival.  A seamless travel experience.",
    "I would to start by saying I do not want points or a travel certificate ( which are always nice) what I really want is to simply provide feedback to better the customer experience. My family sat in seats1A,B, E on New Year's Day. For better or worst you see everything. I was surprised ( and a little disgusted) to see the flight attendant blow her nose no fewer than 15 times during our flight not once washing her hands. If we are lucky, she had a sinus infection, otherwise, she is patient zero when the entire first class cabin has a cold or flew in the next 7-10 days.",
    "The airplanes are too small. My knees are jammed against the seat in front of me. Missed my connection in Denver due to crew availability. Spent and extra 5 hours waiting for the next flight.",
    "I am usually pleased with United. I fly you all 3-6 times a year depending on business meetings etc. My other choice is southwest and that is Bc of their flight direct into Nashville. I do like their baggage policy. It is easier for me to deal with. I mostly fly economy, sometimes basic or plus. I hate having to figure out what I can or can not bring on. Southwest is straightforward. I did get a chance to browse the new subscriptions you all have for economy plus or WiFi. My fiancé and I will look into that Bc it seems really suiting to always have economy plus for the year for around $600 per person. I do wonder if the subscription is prorated. I would not want to subscribe next month and still pay for a full year. A follow up on that would be great."
]
questions_and_comparisons(allrevlist[14])
print(allrevlist[3] + "\n\n")
generate_response(allrevlist[3])
print(allrevlist[1] + "\n\n")
generate_response(allrevlist[1])
print("plane was on time")
generate_response("plane was on time")
